# 目录
在载入、合并和准备数据集后，可能还需要对源数据集进行过滤、变换和分割等操作，本节内容与 ***pandas入门.ipynb*** 中的“6.数据规整”一同作为将用于报告的数据透视表的源数据集重要操作。

数据聚合与分组操作（Data Aggregation and Group Operations）：

1. Groupby机制
    1. 遍历各组
    2. 选择一列或所有列的子集
    3. 使用字典和Series分组
    4. 使用函数分组
    5. 根据索引层级分组
2. 数据聚合
    1. 逐列及多函数应用
    2. 返回不含行索引的聚合数据
3. 应用：通用拆分-应用-联合
    1. 压缩分组键
    2. 分位数与桶分析
    3. 示例：使用指定分组填充缺失值
    4. 随机采样与排列
    5. 示例：分组加权平均和相关性
    6. 示例：逐组线性回归
4. 数据透视表与交叉表
    - 交叉表 -> crosstab

In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 1.Groupby机制（GroupBy Mechanics）

<font color='red'>组操作可分为：拆分(split)-应用(apply)-联合(combine)</font>

- 根据*Series、DataFrame或其他数据结构*的键分离成组（在数据对象特定的轴上操作），分组操作后，一个函数可以应用到各个组中，产生新的值；最后，所有函数的应用结果联合成一个结果对象。

详见 ***pandas入门.ipynb*** 中的“5.数据规整” -> `frame.groupby(level='key').sum()`

分组键可以是多种类型，并且键不一定是完全相同的类型：

- 与需要分组的轴向长度一致的值列表或值数组

- DataFrame的列名的值

- 可以将分组轴向上的值和分组名称相匹配的字典或Series

- 可以在轴索引或索引中的单个标签上调用的函数

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [5]:
# 根据key1标签计算data1列的均值：
grouped = df['data1'].groupby(df['key1'])       # 根据key1列的唯一值进行分组
print(grouped, '\n')
grouped.mean()

key1
a    0.746672
b   -0.537585
Name: data1, dtype: float64

In [7]:
# 在上述数组的基础上，对多列分组键键计算均值
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
print(means, '\n')
means.unstack()

key1  key2
a     one     0.880536
      two     0.478943
b     one    -0.519439
      two    -0.555730
Name: data1, dtype: float64 



key2,one,two
key1,,
a,0.880536,0.478943
b,-0.519439,-0.555730


In [9]:
# 在DataFrame中传入列名作为分组键：
df.groupby('key1').mean()
# size返回一个组大小信息的Series 
print(df.groupby(['key1', 'key2']).size(), '\n')
df.groupby(['key1', 'key2']).mean()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64 



data1     data2
key1 key2                    
a    one   0.880536  1.319920
     two   0.478943  0.092908
b    one  -0.519439  0.281746
     two  -0.555730  0.769023

### ①遍历各分组（Iterating Over Groups）

In [10]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
1    a  two  0.478943  0.092908
4    a  one  1.965781  1.246435
b
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
3    b  two -0.555730  0.769023


In [17]:
# 在多个分组键的情况下，元组中的第一个元素是键值的元组
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

# 默认情况下，groupby在axis=0的轴上分组
print('\n', df.dtypes, '\n')
grouped = df.groupby(df.dtypes, axis=1)

# 计算一行数据块时，将其提取为字典可能会方便些
pieces = dict(list(df.groupby('key1')))
pieces['b']

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
4    a  one  1.965781  1.246435
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.478943  0.092908
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
('b', 'two')
  key1 key2    data1     data2
3    b  two -0.55573  0.769023

 key1      object
key2      object
data1    float64
data2    float64
dtype: object 



,key1,key2,data1,data2
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023


### ②选择一列或所有列的子集（Selecting a Column or Subset of Columns）

In [21]:
# 将从DataFrame创建的GroupBy对象用列名称或列名称数组进行索引时，会产生用于聚合的列子集效果
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]
# 下面为其语法糖：
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

In [22]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped
s_grouped.mean()

key1  key2
a     one     1.319920
      two     0.092908
b     one     0.281746
      two     0.769023
Name: data2, dtype: float64

### ③使用字典和Series分组

In [24]:
# 假设有一组数据以非数组的形式存在
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
print(people, '\n')
# 现在有各列的分组对应关系，并且想把各列按组累加
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

               a         b         c         d         e
Joe     0.560145 -1.265934  0.119827 -1.063512  0.332883
Steve  -2.359419 -0.199543 -1.541996 -0.970736 -1.307030
Wes     0.286350       NaN       NaN  0.331286  1.349742
Jim     0.069877  0.246674 -0.011862  1.004812  1.327195
Travis -0.919262 -1.549106  0.022185  0.758363 -0.660524 



In [26]:
# 可以将字典构造传递给groupby的数组
by_column = people.groupby(mapping, axis=1)
print(by_column.sum(), '\n')

# Series也有相同功能
map_series = pd.Series(mapping)
print(map_series, '\n')
people.groupby(map_series, axis=1).count()

            blue       red
Joe    -0.943685 -0.372906
Steve  -2.512731 -3.865992
Wes     0.331286  1.636092
Jim     0.992950  1.643745
Travis  0.780548 -3.128892 

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object 



,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### ④使用函数分组（Grouping with Functions）

作为分组键传递的函数将会按照每个索引值调用一次，同时返回值会被用作分组名称。（也就是说，省去了转换为字典或Series对象然后再调用函数联合结果这一步）

In [27]:
# 根据人名长度来分组（len根据行标签字符串长度）
people.groupby(len).sum()

,a,b,c,d,e
3,0.916372,-1.019260,0.107966,0.272585,3.009820
5,-2.359419,-0.199543,-1.541996,-0.970736,-1.307030
6,-0.919262,-1.549106,0.022185,0.758363,-0.660524


In [28]:
# 函数与数组、字典或Series对象混合时，所有的对象都会在内部转换为数组
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  0.286350 -1.265934  0.119827 -1.063512  0.332883
  two  0.069877  0.246674 -0.011862  1.004812  1.327195
5 one -2.359419 -0.199543 -1.541996 -0.970736 -1.307030
6 two -0.919262 -1.549106  0.022185  0.758363 -0.660524

### ⑤根据索引层级分组（Grouping by Index Levels）

In [29]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.862580 -0.010032  0.050009  0.670216  0.852965
1     -0.955869 -0.023493 -2.304234 -0.652469 -1.218302
2     -1.332610  1.074623  0.723642  0.690002  1.001543
3     -0.503087 -0.622274 -0.921169 -0.726213  0.222896

In [30]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 2.数据聚合（Data Aggregation）

聚合是指所有数组产生标量值的数据转换过程。

<font color='red'>优化的groupby方法：</font>

|函数名|描述|
|:--:|:--:|
|count|分组中的非Na值数量|
|sum|非NA值的累和|
|mean|非NA值的均值|
|median|非NA值的算术中位数|
|std、var|无偏的（n-1分母）标准差和方差|
|min、max|非NA值的最小值、最大值|
|prod|非NA值的乘积|
|first、last|非NA值的第一个和最后一个值|

In [32]:
print(df)
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

  key1 key2     data1     data2
0    a  one -0.204708  1.393406
1    a  two  0.478943  0.092908
2    b  one -0.519439  0.281746
3    b  two -0.555730  0.769023
4    a  one  1.965781  1.246435


key1
a    1.668413
b   -0.523068
Name: data1, dtype: float64

In [33]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

/var/folders/v0/q2247qvj6nx2z3kxhc2_snyw0000gn/T/ipykernel_3064/663192242.py:3: FutureWarning: ['key2'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped.agg(peak_to_peak)


,data1,data2
key1,,
a,2.170488,1.300498
b,0.036292,0.487276


In [34]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.746672  1.109736 -0.204708  0.137118  0.478943  1.222362   
b      2.0 -0.537585  0.025662 -0.555730 -0.546657 -0.537585 -0.528512   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.965781   3.0  0.910916  0.712217  0.092908  0.669671  1.246435   
b    -0.519439   2.0  0.525384  0.344556  0.281746  0.403565  0.525384   

                          
           75%       max  
key1                      
a     1.319920  1.393406  
b     0.647203  0.769023

### ①逐列及多函数应用（Column-Wise and Multiple Function Application）

In [35]:
# 在pandas入门的第五节中使用了tips比例：
tips = pd.read_csv('examples/tips.csv')
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
print(tips[:6], '\n')

   total_bill   tip smoker  day    time  size   tip_pct
0       16.99  1.01     No  Sun  Dinner     2  0.059447
1       10.34  1.66     No  Sun  Dinner     3  0.160542
2       21.01  3.50     No  Sun  Dinner     3  0.166587
3       23.68  3.31     No  Sun  Dinner     2  0.139780
4       24.59  3.61     No  Sun  Dinner     4  0.146808
5       25.29  4.71     No  Sun  Dinner     4  0.186240 



In [39]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')     # 将函数名以字符串的形式传递
grouped_pct.agg(['mean', 'std', peak_to_peak])  # 传递函数名列表

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [40]:
# 如果传递的是(name, func)元组的列表，那么元组的第一个元素名称将作为DataFrame的列名
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [46]:
# 指定应用到所有列上的函数列表或每一列上要应用的不同函数 -> 列表的分别索引
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

/var/folders/v0/q2247qvj6nx2z3kxhc2_snyw0000gn/T/ipykernel_3064/3838903792.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped['tip_pct', 'total_bill'].agg(functions)


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [49]:
# 每一列应用不同函数，并且输出时将函数名改成指定名称
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

/var/folders/v0/q2247qvj6nx2z3kxhc2_snyw0000gn/T/ipykernel_3064/3909980601.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct', 'total_bill'].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [50]:
# 每一列应用不同函数的另一种实现方式 -> 字典
grouped.agg({'tip' : np.max, 'size' : 'sum'})
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### ②返回不含行索引的聚合数据（Returning Aggregated Data Without Row Indexes）

In [51]:
# 聚合数据返回时都是带有索引的，但有时候需要禁用索引，传递参数 as_index=False
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 3.应用：通用拆分-应用-联合（Apply: General split-apply-combine）

In [52]:
# 在之前的小费数据集中，选出小费百分比最高的五组
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [54]:
# 按'smoker'进行分组 -> top函数在每一层上都调用，之后使用pandas.concat将函数结果粘贴在一起，并使用分组名作为各组的标签
tips.groupby('smoker').apply(top)
# 传递多个参数到apply()函数中
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [ ]:
result = tips.groupby('smoker')['tip_pct'].describe()
print(result, '\n')
result.unstack('smoker')

""" 在GroupBy对象的内部，调用像describe方法时，实际是以下代码的缩写：
f = lambda x: x.describe()
grouped.apply(f)s
 """

### ①压缩分组键（Suppressing the Group Keys）

In [57]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### ②分位数与桶分析（Quantile and Bucket Analysis）

- `pd.cut()`

- `pd.qcut(label=True)`

In [58]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)      # 平均分成四个区间
quartiles[:10]

0    (-1.23, 0.489]
1    (-1.23, 0.489]
2    (0.489, 2.208]
3    (-1.23, 0.489]
4    (0.489, 2.208]
5    (0.489, 2.208]
6    (-1.23, 0.489]
7    (0.489, 2.208]
8    (-1.23, 0.489]
9    (0.489, 2.208]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-2.956, -1.23] < (-1.23, 0.489] < (0.489, 2.208] < (2.208, 3.928]]

In [59]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
grouped = frame.data2.groupby(quartiles)        # 对data2下的值应用四段分区
grouped.apply(get_stats).unstack()              # 对每个分区都调用函数，并返回按轴0排列的数据

,min,max,count,mean
data1,,,,
"(-2.956, -1.23]",-3.399312,1.670835,94.0,-0.043940
"(-1.23, 0.489]",-2.989741,3.260383,599.0,-0.011459
"(0.489, 2.208]",-3.745356,2.954439,296.0,0.092068
"(2.208, 3.928]",-1.929776,1.765640,11.0,0.055158


In [60]:
# 返回分位数数值
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-3.399312,1.670835,100.0,-0.022372
1,-2.153545,2.628441,100.0,-0.045403
2,-2.925113,2.527939,100.0,-0.085406
3,-2.315555,3.260383,100.0,0.073851
4,-2.047939,2.074345,100.0,-0.120305
5,-2.989741,2.184810,100.0,0.036851
6,-2.084231,2.458842,100.0,0.001262
7,-3.056990,2.954439,100.0,-0.006403
8,-3.745356,2.735527,100.0,0.120793


### ③示例：使用指定分组填充缺失值（Example: Filling Missing Values with Group-Specific Values）

In [61]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s
s.fillna(s.mean())

0    0.127265
1    1.141487
2    0.127265
3    0.125036
4    0.127265
5   -0.884727
dtype: float64

In [65]:
# 需要填充值按组来变换，下面为将美国分为东部和西部地区的样本数据

# 设置初始元素与分组备份表
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4     # 包含了四个['East']和四个['West']中元素的备份列表
data = pd.Series(np.random.randn(8), index=states)
print(data, '\n')

# 现将数据中的一些值设置为缺失值
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

# 用分组的平均值填充NA值
fill_mean = lambda g: g.fillna(g.mean())
print(data.groupby(group_key).apply(fill_mean), '\n')

# 在另一种情况中可能已经预定了缺失值的填充值，那么可以使用每个分组都具有的'name'属性
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio          0.592436
New York      0.728461
Vermont       0.087170
Florida      -0.057191
Oregon       -1.690798
Nevada        0.310821
California   -1.747370
Idaho         1.308413
dtype: float64 

Ohio          0.592436
New York      0.728461
Vermont       0.421235
Florida      -0.057191
Oregon       -1.690798
Nevada       -1.719084
California   -1.747370
Idaho        -1.719084
dtype: float64 



Ohio          0.592436
New York      0.728461
Vermont       0.500000
Florida      -0.057191
Oregon       -1.690798
Nevada       -1.000000
California   -1.747370
Idaho        -1.000000
dtype: float64

### ④示例：随机采样与排列（Example: Random Sampling and Permutation）

Series中的`sample()`

In [68]:
# 以一种英式扑克牌为例

# Hearts, Spades, Clubs, Diamonds（红桃、黑桃、梅花、方块）
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [70]:
# 从副牌中随机抽取五张牌
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

6D      6
4H      4
3C      3
5D      5
10C    10
dtype: int64

In [71]:
# 假设要想从每个花色中随机抽取两张牌（花色是牌名的最后两个字符）
get_suit = lambda card: card[-1]    # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

C  7C     7
   8C     8
D  9D     9
   3D     3
H  5H     5
   2H     2
S  8S     8
   JS    10
dtype: int64

### ⑤示例：分组加权平均和相关性（Example: Group Weighted Average and Correlation）

In [72]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,-0.082186,0.941527
1,a,1.343546,0.035548
2,a,-1.777409,0.453110
3,a,1.246595,0.680759
4,b,-0.843458,0.258831
5,b,0.247391,0.835434
6,b,-0.521944,0.125923
7,b,0.412443,0.661177


In [73]:
# 通过'category'分组后加权平均
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a    0.006466
b    0.103828
dtype: float64

In [74]:
# 考虑从雅虎财经上获得的数据集

close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True,
                       index_col=0)
close_px.info()
close_px[-4:]

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [77]:
# 计算一个DataFrame，它包含标普指数（SPX）每日收益的年度相关性（通过百分比变化得到）

spx_corr = lambda x: x.corrwith(x['SPX'])
rets = close_px.pct_change().dropna()       # pct_change()计算百分比
# 按年对百分比变化进行分组
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [78]:
# 计算微软和苹果的年度相关性
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### ⑥示例：逐组线性回归（Example: Group-Wise Linear Regression）

对每个数据执行普通最小二乘(OLS)回归

In [80]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 4.数据透视表与交叉表（Pivot Tables and Cross-Tabulation）

DataFrame中的 `pivot_table`  <font color='red'>参数如下：</font>

|选项名|描述|
|:--:|:--:|
|values|需要聚合的列名，默认情况下聚合所有数值行的列|
|index|在结果透视表的行上进行分组的列名或其他分组键|
|columns|在结果透视表的列上进行分组的列名或其他分组键|
|aggfunc|聚合函数或函数列表（默认情况下是'mean'）；可是groupby上下文的任意有效函数|
|fill_value|在结果表中替换缺失值的值|
|dropna|如果为True，将不含所有条目均为NA的列|
|margins|添加行/列小计和总计（默认为False）|

In [81]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [82]:
# 假设只在tip_pct和size上聚合，并根据time分组
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [83]:
# 传递参数margins=True来扩充这表来包含部分总计
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                 aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [84]:
# 要使用不同的聚合函数时，将函数传递给 aggfunc
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
...                      ...       ...       ...       ...
Lunch  3    Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

[21 rows x 4 columns]

### 交叉表（Cross-Tabulations）-> Crosstab

In [86]:
from io import StringIO
data = """\
Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [88]:
# crosstab的前两个参数可以是数组、Series或数组的列表
pd.crosstab(data.Nationality, data.Handedness, margins=True)
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

In [89]:
# 设置最大可显示行数
pd.options.display.max_rows = PREVIOUS_MAX_ROWS